In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import classification_report
cudnn.benchmark = True
plt.ion()

In [6]:
data_transforms = transforms.Compose([
     transforms.Resize((300, 400)), 
                                 transforms.ToTensor()
])
image_datasets = {x: datasets.ImageFolder(os.path.join('frames', x),data_transforms)
                  for x in ['train','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4) for x in ['train','test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','test']}
class_names = image_datasets['train'].classes
def train_model(model, criterion, optimizer, scheduler):
    num_epochs=25
    best_model_wts = copy.deepcopy(model.state_dict())
    for e in range(num_epochs):
        print(f'Epoch {e}/{num_epochs}')
        for phase in ['train','test']:
            if phase == 'train':
                model.train()  
            else:
                model.eval() 
            loss_value = 0.0
            count = 0
            for inputs, labels in dataloaders[phase]:
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    x, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                loss_value+=loss.item()*inputs.size(0)
                count+=torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = loss_value / dataset_sizes[phase]
            epoch_acc = count.double() / dataset_sizes[phase]
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model



In [9]:
pre_model = models.resnet50(pretrained=True)
pre_model.fc = nn.Linear(pre_model.fc.in_features, 3)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(pre_model.parameters(), lr=0.001, momentum=0.9)
pre_model = train_model(pre_model, criterion, optimizer_ft, lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1))

Epoch 0/24
----------
train Loss: 0.4552 Acc: 0.8212
test Loss: 0.8313 Acc: 0.6934

Epoch 1/24
----------
train Loss: 0.2730 Acc: 0.9010
test Loss: 0.7612 Acc: 0.7339

Epoch 2/24
----------
train Loss: 0.1612 Acc: 0.9434
test Loss: 1.0070 Acc: 0.7518

Epoch 3/24
----------
train Loss: 0.0847 Acc: 0.9731
test Loss: 0.5971 Acc: 0.7282

Epoch 4/24
----------
train Loss: 0.0410 Acc: 0.9873
test Loss: 0.9655 Acc: 0.7257

Epoch 5/24
----------
train Loss: 0.0186 Acc: 0.9958
test Loss: 1.0641 Acc: 0.7494

Epoch 6/24
----------
train Loss: 0.0245 Acc: 0.9941
test Loss: 1.0968 Acc: 0.6775

Epoch 7/24
----------
train Loss: 0.0088 Acc: 0.9980
